This notebook contains combination combination of our work on the LTN. It consinsts of the most important findings and LTN implementation in a structured way. This is not all the code that was developed during the project and only the most interesting parts. More code is available on the development branch in few separate notebooks. 

# Import of libraries and data preparation

In [1]:
import torch
import pandas as pd
import ltn
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt


## Loading the dataset

In [3]:
ds_l = pd.read_csv('src\data\Stud_E-mobility_data_staticLimit.csv')

Main dataset with no labeling

In [4]:
ds_l.columns

Index(['_time', 'GARAGE_EXTERNAL_POWER', 'DEMAND_LIMIT',
       'DEMAND_LIMIT_INDICATOR', 'BATTERY_SOC', 'BATTERY_DISCHARGE_POWER',
       'BATTERY_CHARGED_ENERGY', 'BATTERY_DISCHARGED_ENERGY', 'PV_POWER',
       'PV_ENERGY', 'WALLBOX_ALPHA_ENERGY', 'WALLBOX_ALPHA_POWER',
       'WALLBOX_1_ENERGY', 'WALLBOX_1_POWER', 'WALLBOX_2_ENERGY',
       'WALLBOX_2_POWER', 'WALLBOX_3_ENERGY', 'WALLBOX_3_POWER',
       'WALLBOX_A_ENERGY', 'WALLBOX_A_POWER', 'WALLBOX_B_ENERGY',
       'WALLBOX_B_POWER', 'WALLBOX_C_ENERGY', 'WALLBOX_C_POWER',
       'WALLBOX_FASTCHARGER_ENERGY', 'WALLBOX_FASTCHARGER_POWER'],
      dtype='object')

In [5]:
ds = ds_l[['GARAGE_EXTERNAL_POWER','DEMAND_LIMIT',
       'BATTERY_SOC', 'BATTERY_DISCHARGE_POWER',
       'WALLBOX_FASTCHARGER_POWER', 'PV_POWER'
    ]]

Ground truth labeling (whole dataset)

In [23]:
gt_ds = ds.copy()

In [24]:
def label_charging(row):
    if row["BATTERY_SOC"] > 80:
        return "Fully Covered by Local Battery"
    elif 40 <= row["BATTERY_SOC"] < 80:
        if row["GARAGE_EXTERNAL_POWER"] > row["DEMAND_LIMIT"]:
            return "Partially Covered by Local Battery"
        else:
            return "Battery Charged from Grid"
    elif 15 <= row["BATTERY_SOC"] <= 40:
        if row["GARAGE_EXTERNAL_POWER"] > row["DEMAND_LIMIT"]:
            return "Partially Covered by Local Battery"
        else:
            return "Battery Charged from Grid"
    # elif row["BATTERY_SOC"] < 15:
    elif row["BATTERY_SOC"] < 15:
        return "Battery Discharge Stopped due to Battery Health"
    else:
        print(row["BATTERY_SOC"])
        print(row["GARAGE_EXTERNAL_POWER"])
        return "Unknown"

# Apply the labeling function to create the new column "DRAWN_FROM"
gt_ds["DRAWN_FROM"] = gt_ds.apply(label_charging, axis=1)

In [25]:
gt_ds['DRAWN_FROM'].value_counts()

DRAWN_FROM
Battery Charged from Grid                          54783
Partially Covered by Local Battery                  4457
Battery Discharge Stopped due to Battery Health      202
Name: count, dtype: int64

Small labeled dataset (only the part that follows GT)

In [14]:
delta = 0.5 # Tolerance for the power limit
SOC_less_15 = gt_ds[(gt_ds["BATTERY_SOC"]<=15) & (gt_ds["BATTERY_DISCHARGE_POWER"]<=0)]
SOC_less_40_1 = gt_ds[(gt_ds["BATTERY_SOC"]>15) &(gt_ds["BATTERY_SOC"]<40) & (gt_ds["GARAGE_EXTERNAL_POWER"]<50) & (gt_ds["BATTERY_DISCHARGE_POWER"]<0)]
SOC_less_40_2 = gt_ds[(gt_ds["BATTERY_SOC"]>15) &(gt_ds["BATTERY_SOC"]<40) & (gt_ds["GARAGE_EXTERNAL_POWER"]<=(50+delta)) & ((50-delta)<=gt_ds["GARAGE_EXTERNAL_POWER"]) & (gt_ds["BATTERY_DISCHARGE_POWER"]>=0)]
SOC_more_40 = gt_ds[(gt_ds["BATTERY_SOC"]>=40) & (gt_ds["BATTERY_DISCHARGE_POWER"]>=0)]

In [16]:
gt_ds_small = pd.concat([SOC_less_15, SOC_less_40_1, SOC_less_40_2, SOC_more_40], ignore_index=True)
gt_ds_small = gt_ds_small.drop_duplicates()
print(f"Percentage of dataset, that is kept: {len(gt_ds_small)/len(gt_ds)*100}%")

Percentage of dataset, that is kept: 16.313381110998957%


Getting features and target

In [26]:
features = gt_ds.drop(['DEMAND_LIMIT','GARAGE_EXTERNAL_POWER', 'DRAWN_FROM'], axis=1)
target = gt_ds['DRAWN_FROM']

In [28]:
encoder = LabelEncoder()
en_targ = encoder.fit_transform(target)

In [29]:
# Print classes and their labels
for label, original_class in enumerate(encoder.classes_):
    print(f'Original Class: "{original_class}" is encoded as {label}')

Original Class: "Battery Charged from Grid" is encoded as 0
Original Class: "Battery Discharge Stopped due to Battery Health" is encoded as 1
Original Class: "Partially Covered by Local Battery" is encoded as 2


# Logic Tensor Network

## Simple LTN classification model

Split data into train and test, encode the labels into ltn Constants

In [118]:

features_train, features_test, target_train, target_test = train_test_split(features, en_targ, test_size=0.2, random_state=42)
features_train = torch.tensor(features_train.to_numpy()).float()
features_test = torch.tensor(features_test.to_numpy()).float()

In [119]:
l_A = ltn.Constant(torch.tensor([1, 0, 0]))
l_B = ltn.Constant(torch.tensor([0, 1, 0]))
l_C = ltn.Constant(torch.tensor([0, 0, 1]))

### Models implementation and main predicate

We need two separated models because we need both logits and probabilities. Logits are used to compute the classification accuracy, while probabilities are interpreted as truth values to compute the satisfaction level of the knowledge base.

In [40]:
class MLP(torch.nn.Module):
    def __init__(self, layer_sizes=(4, 100, 52, 52, 3)):
        super(MLP, self).__init__()
        self.elu = torch.nn.ELU()
        self.dropout = torch.nn.Dropout(0.2)
        self.linear_layers = torch.nn.ModuleList([torch.nn.Linear(layer_sizes[i - 1], layer_sizes[i])
                                                  for i in range(1, len(layer_sizes))])

    def forward(self, x, training=False):
        for layer in self.linear_layers[:-1]:
            x = self.elu(layer(x))
            if training:
                x = self.dropout(x)
        logits = self.linear_layers[-1](x)
        return logits

class LogitsToPredicate(torch.nn.Module):
    """
    This model has inside a logits model, that is a model which compute logits for the classes given an input example x.
    The idea of this model is to keep logits and probabilities separated. The logits model returns the logits for an example,
    while this model returns the probabilities given the logits model.

    In particular, it takes as input an example x and a class label l. It applies the logits model to x to get the logits.
    Then, it applies a softmax function to get the probabilities per classes. Finally, it returns only the probability related
    to the given class l.
    """
    def __init__(self, logits_model):
        super(LogitsToPredicate, self).__init__()
        self.logits_model = logits_model
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x, l, training=False):
        logits = self.logits_model(x, training=training)
        probs = self.softmax(logits)
        out = torch.sum(probs * l, dim=1)
        return out


In [41]:
class DataLoader(object):
    def __init__(self,
                 data,
                 labels,
                 batch_size=1,
                 shuffle=True):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.unique_labels = np.unique(labels) 

    def __len__(self):
        return int(np.ceil(self.data.shape[0] / self.batch_size))

    def __iter__(self):
        indices_per_class = {label: np.where(self.labels == label)[0] for label in self.unique_labels}

        samples_per_class = self.batch_size // len(self.unique_labels)

        for _ in range(len(self)):
            batch_indices = []

            for label in self.unique_labels:
                class_indices = np.random.choice(indices_per_class[label], size=samples_per_class, replace=True)
                batch_indices.extend(class_indices)


            if len(batch_indices) < self.batch_size:
                extra_indices = np.random.choice(np.arange(len(self.labels)), size=self.batch_size - len(batch_indices))
                batch_indices.extend(extra_indices)

            if self.shuffle:
                np.random.shuffle(batch_indices)

            yield self.data[batch_indices], self.labels[batch_indices]

Here we use LTN to define a predicate that will make use of our model

In [42]:
mlp = MLP()
P = ltn.Predicate(LogitsToPredicate(mlp))

### Utilities used by LTN, logistic operations, satisfaction and accuracy retrival

In [43]:
And = ltn.Connective(ltn.fuzzy_ops.AndProd())
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())
Exists = ltn.Quantifier(ltn.fuzzy_ops.AggregPMean(p=2), quantifier="e")
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
SatAgg = ltn.fuzzy_ops.SatAgg()

In [44]:
def compute_sat_level(loader):
    mean_sat = 0
    for data, labels in loader:
        x_A = ltn.Variable("x_A", data[labels == 0])
        x_B = ltn.Variable("x_B", data[labels == 1])
        x_C = ltn.Variable("x_C", data[labels == 2])
        mean_sat += SatAgg(
            Forall(x_A, P(x_A, l_A)),
            Forall(x_B, P(x_B, l_B)),
            Forall(x_C, P(x_C, l_C))
        )
    mean_sat /= len(loader)
    return mean_sat

def compute_accuracy(loader):
    mean_accuracy = 0.0
    for data, labels in loader:
        predictions = mlp(data).detach().numpy()
        predictions = np.argmax(predictions, axis=1)
        mean_accuracy += accuracy_score(labels, predictions)

    return mean_accuracy / len(loader)


In [120]:

train_loader = DataLoader(features_train, target_train, 64, shuffle=True)
test_loader = DataLoader(features_test, target_test, 64, shuffle=False)

### Learning 

In [46]:
optimizer = torch.optim.Adam(P.parameters(), lr=0.001)

for epoch in range(120):
    train_loss = 0.0
    for batch_idx, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        # we ground the variables with current batch data
        x_A = ltn.Variable("x_A", data[labels == 0]) 
        x_B = ltn.Variable("x_B", data[labels == 1]) 
        x_C = ltn.Variable("x_C", data[labels == 2]) 

        # calculating the satisfaction level of the knowledge base, used in guided learning
        sat_agg = SatAgg(
            Forall(x_A, P(x_A, l_A, training=True)),
            Forall(x_B, P(x_B, l_B, training=True)),
            Forall(x_C, P(x_C, l_C, training=True))
        )
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)

    # we print metrics every 20 epochs of training
    # Here is an example of simple quering of the logistic formulas
    if epoch % 20 == 0:
        print(" epoch %d | loss %.4f | Train Sat %.3f | Test Sat %.3f | Train Acc %.3f | Test Acc %.3f"
              %(epoch, train_loss, compute_sat_level(train_loader), compute_sat_level(test_loader),
                    compute_accuracy(train_loader), compute_accuracy(test_loader)))

 epoch 0 | loss 0.2683 | Train Sat 0.770 | Test Sat 0.771 | Train Acc 0.928 | Test Acc 0.929
 epoch 20 | loss 0.2072 | Train Sat 0.792 | Test Sat 0.782 | Train Acc 0.943 | Test Acc 0.941
 epoch 40 | loss 0.2096 | Train Sat 0.793 | Test Sat 0.801 | Train Acc 0.944 | Test Acc 0.943
 epoch 60 | loss 0.2055 | Train Sat 0.799 | Test Sat 0.798 | Train Acc 0.943 | Test Acc 0.944
 epoch 80 | loss 0.2040 | Train Sat 0.795 | Test Sat 0.785 | Train Acc 0.944 | Test Acc 0.941
 epoch 100 | loss 0.2022 | Train Sat 0.802 | Test Sat 0.809 | Train Acc 0.949 | Test Acc 0.949


Final result

In [48]:
print(" epoch %d | loss %.4f | Train Sat %.3f | Test Sat %.3f | Train Acc %.3f | Test Acc %.3f"
              %(epoch, train_loss, compute_sat_level(train_loader), compute_sat_level(test_loader),
                    compute_accuracy(train_loader), compute_accuracy(test_loader)))

# epoch 119 | loss 0.2018 | Train Sat 0.801 | Test Sat 0.808 | Train Acc 0.948 | Test Acc 0.950

 epoch 119 | loss 0.2018 | Train Sat 0.801 | Test Sat 0.808 | Train Acc 0.948 | Test Acc 0.950


For that model already after 20 epochs we can see Test Acc == 0.941 and the rule satisfaction on Test == 0.782

## Rules satisfaction presentation
Rules satisfaction allows for a simple verification of some believes, theories as to the data (or simply verification of what we thing could be a ,,rule'' as to the system behaviour). This, to some extend, is usually possible with data analysis (in case of simple rules), but as shown later on - plays a crucial role in finding a treshold and allows for a things outside of the scope of the simple data analysis.

In [68]:
testing_set = DataLoader(torch.tensor(features.to_numpy()).float(), torch.tensor(en_targ), 64, shuffle=False)

In [69]:
def compute_sat_level_phi(loader, phi):
    sat_values = []
    for features, labels in loader:
        for feature, label in zip(features, labels):
            sat_values.append(phi(feature, label).value)
    mean_sat = torch.mean(torch.stack(sat_values))
    return mean_sat

First let's check the satisfaction level of the rule from the GT 
- If SOC < 15 than Battery Discharge Stopped due to Battery Health 

In [82]:
def discharge_stopped(tensor):
    return tensor <= 0

def Battery_SOC_small(tensor):
    return tensor < 15 

Battery_SOC_small = ltn.Predicate(None, Battery_SOC_small)
discharge_stopped = ltn.Predicate(None, discharge_stopped)

In [83]:

def phi(features, label):
    sc = features[0].view(-1,1)
    bp = features[1].view(-1,1)

    bp = ltn.Variable("bp", bp)
    s = ltn.Variable("s", sc)
    return Forall(s, Implies(Battery_SOC_small(s), discharge_stopped(bp)), p=5)



In [84]:
compute_sat_level_phi(testing_set, phi)

tensor(0.9096)

In [107]:
def partially_covered_by_local_battery(tensor_soc, tensor_garage_external_power, tensor_demand_limit):
    condition1 = ((tensor_soc >= 40) & (tensor_soc < 80)) | ((tensor_soc >= 15) & (tensor_soc <= 40))
    condition2 = tensor_garage_external_power > tensor_demand_limit
    return condition1 & condition2

def battery_charged_from_grid(tensor_soc, tensor_garage_external_power, tensor_demand_limit):
    condition1 = ((tensor_soc >= 40) & (tensor_soc < 80)) | ((tensor_soc >= 15) & (tensor_soc <= 40))
    condition2 = tensor_garage_external_power <= tensor_demand_limit
    return condition1 & condition2

partially_covered_by_local_battery_pred = ltn.Predicate(None, partially_covered_by_local_battery)
battery_charged_from_grid_pred = ltn.Predicate(None, battery_charged_from_grid)


In [108]:
def mutual_exclusivity_formula_batch(soc_batch, garage_external_power_batch, demand_limit_batch):
    soc_var = ltn.Variable("soc", soc_batch)
    garage_external_power_var = ltn.Variable("garage_external_power", garage_external_power_batch)
    demand_limit_var = ltn.Variable("demand_limit", demand_limit_batch)

    return Forall(
        [soc_var, garage_external_power_var, demand_limit_var],
        Not(
            And(
                partially_covered_by_local_battery_pred(soc_var, garage_external_power_var, demand_limit_var),
                battery_charged_from_grid_pred(soc_var, garage_external_power_var, demand_limit_var)
            )
        )
    ).value


In [109]:
def compute_sat_level_in_chunks(batch_size=100):
    soc = torch.tensor(ds_l["BATTERY_SOC"].to_numpy())
    garage_external_power = torch.tensor(ds_l["GARAGE_EXTERNAL_POWER"].to_numpy())
    demand_limit = torch.tensor(ds_l["DEMAND_LIMIT"].to_numpy())
    num_samples = len(soc)
    sat_values = []
    
    for start_idx in range(0, num_samples, batch_size):
        end_idx = min(start_idx + batch_size, num_samples)
        
        soc_batch = soc[start_idx:end_idx]
        garage_external_power_batch = garage_external_power[start_idx:end_idx]
        demand_limit_batch = demand_limit[start_idx:end_idx]
        
        sat_value = mutual_exclusivity_formula_batch(soc_batch, garage_external_power_batch, demand_limit_batch)
        sat_values.append(sat_value)
    
    mean_sat = torch.mean(torch.stack(sat_values))
    return mean_sat


In [110]:
def mutual_exclusivity_formula(ds_l, batch_size=100):
    # Convert dataset columns to tensors
    soc = torch.tensor(ds_l["BATTERY_SOC"].to_numpy())
    garage_external_power = torch.tensor(ds_l["GARAGE_EXTERNAL_POWER"].to_numpy())
    demand_limit = torch.tensor(ds_l["DEMAND_LIMIT"].to_numpy())
    
    # Compute satisfaction level in chunks
    return compute_sat_level_in_chunks(soc, garage_external_power, demand_limit, batch_size)


In [111]:
compute_sat_level_in_chunks()

tensor(0.9998)

In [112]:
class ThresholdModel(torch.nn.Module):
    """
    This model returns a single value (threshold) given a set of features. 
    """
    def __init__(self, input_size, hidden_size):
        super(ThresholdModel, self).__init__()
        self.elu = torch.nn.ELU()
        self.linear1 = torch.nn.Linear(input_size, hidden_size)
        self.linear2 = torch.nn.Linear(hidden_size, 1)

    def forward(self, x):
        """
        Method which defines the forward phase of the neural network.

        :param x: the features of the example
        :return: threshold for example x
        """
        x = self.elu(self.linear1(x))
        out = self.linear2(x)
        return out


In [117]:
tmodel = ltn.Function(ThresholdModel(input_size=1, hidden_size=8))
mlp = MLP()
P = ltn.Predicate(LogitsToPredicate(mlp))

In [115]:
def Battery_SOC_smaller(tensor, tr):
    return tensor <= tr

Battery_SOC_smaller = ltn.Predicate(None, Battery_SOC_smaller)

In [114]:
epsilon = 1e-7 
alpha = 0.01
Battery_SOC_smaller2 = ltn.Predicate(func=lambda tensor, tr: torch.exp(-alpha * torch.sqrt(torch.sum(torch.square(tr - tensor + epsilon), dim=1))))
alpha = 0.01
Battery_SOC_larger2 = ltn.Predicate(func=lambda tensor, tr: torch.exp(-alpha * torch.sqrt(torch.sum(torch.square(tensor - tr + epsilon), dim=1))))

In [121]:
optimizer = torch.optim.Adam(list(P.parameters()) + list(tmodel.parameters()), lr=0.001)

for epoch in range(120):
    train_loss = 0.0
    for batch_idx, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # we ground the variables with current batch data
        x_A = ltn.Variable("x_A", data[labels == 0]) # class A examples
        x_B = ltn.Variable("x_B", data[labels == 1]) # class B examples
        x_C = ltn.Variable("x_C", data[labels == 2]) # class C examples

        x_B_SOC = ltn.Variable("x_B_SOC", data[labels == 1][:, 0])
        x_C_SOC = ltn.Variable("x_C_SOC", data[labels == 2][:, 0])
        x_A_SOC = ltn.Variable("x_A_SOC", data[labels == 0][:, 0])

        tresh1 = tmodel(x_B_SOC)
        tresh2 = tmodel(x_A_SOC)
        
        tresh = ltn.Variable("tre1", tresh1.value)
        tresh2 = ltn.Variable("tre2", tresh2.value)
        
        sat_agg = SatAgg(    
            # there is a treshold such that SOC is smaller than treshold 
            Forall(x_A, P(x_A, l_A, training=True)),
            Forall(x_B, P(x_B, l_B, training=True)),
            Forall(x_C, P(x_C, l_C, training=True)),
            Exists(tresh, Forall(x_B_SOC, Battery_SOC_smaller2(x_B_SOC, tresh), p=5), p=5),
            Exists(tresh, Forall(x_C_SOC, Battery_SOC_larger2(x_C_SOC, tresh), p=5) ,p=5),
            Exists(tresh2, Forall(x_A_SOC, Battery_SOC_smaller2(x_A_SOC, tresh2), p=5) ,p=5),
            Exists(tresh2, Forall(x_C_SOC, Battery_SOC_larger2(x_C_SOC, tresh2), p=5) ,p=5),
        )
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)

    # we print metrics every 20 epochs of training
    if epoch % 20 == 0:
        print(" epoch %d | loss %.4f | Train Sat %.3f | Test Sat %.3f | Train Acc %.3f | Test Acc %.3f"
              %(epoch, train_loss, compute_sat_level(train_loader), compute_sat_level(test_loader),
                    compute_accuracy(train_loader), compute_accuracy(test_loader)))
        print("Lower treshold 1:", tresh.value.mean(), Forall(x_B_SOC, Battery_SOC_smaller(x_B_SOC, tresh)).value.mean())
        print("Lower treshold 2:", tresh2.value.mean(), Forall(x_A_SOC, Battery_SOC_smaller(x_A_SOC, tresh2)).value.mean())

 epoch 0 | loss 0.2524 | Train Sat 0.770 | Test Sat 0.774 | Train Acc 0.928 | Test Acc 0.929
Lower treshold 1: tensor(14.7386, grad_fn=<MeanBackward0>) tensor(0.8231)
Lower treshold 2: tensor(37.4891, grad_fn=<MeanBackward0>) tensor(0.1571)
 epoch 20 | loss 0.1650 | Train Sat 0.797 | Test Sat 0.800 | Train Acc 0.943 | Test Acc 0.942
Lower treshold 1: tensor(23.4130, grad_fn=<MeanBackward0>) tensor(0.8875)
Lower treshold 2: tensor(37.4974, grad_fn=<MeanBackward0>) tensor(0.0021)
 epoch 40 | loss 0.1608 | Train Sat 0.800 | Test Sat 0.805 | Train Acc 0.945 | Test Acc 0.946
Lower treshold 1: tensor(22.1497, grad_fn=<MeanBackward0>) tensor(0.8366)
Lower treshold 2: tensor(38.0251, grad_fn=<MeanBackward0>) tensor(0.)
 epoch 60 | loss 0.1617 | Train Sat 0.806 | Test Sat 0.808 | Train Acc 0.947 | Test Acc 0.948
Lower treshold 1: tensor(25.5256, grad_fn=<MeanBackward0>) tensor(0.9999)
Lower treshold 2: tensor(37.0858, grad_fn=<MeanBackward0>) tensor(0.0230)
 epoch 80 | loss 0.1598 | Train Sat 0